In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da página
url = "https://fbref.com/pt/comps/11/cronograma/Serie-A-Resultados-e-Calendarios"
# Faz a requisição para a página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Função para extrair a tabela da página
def extrair_tabela(rows):
    headers = []
    data = []
    
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if len(headers) == 0:  # A primeira linha será o cabeçalho
            headers = [col.get_text
                       (strip=True) for col in cols]
        else:  # Demais linhas são dados
            data.append([col.get_text(strip=True) for col in cols])
    
    # Cria o DataFrame
    return pd.DataFrame(data, columns=headers)

# Encontra o container da tabela
table_container = soup.find('div', {'class': 'table_container'})
rows = table_container.find_all('tr')

# Extrai a tabela
tabela = extrair_tabela(rows)

# Encontra a coluna "Relatório da Partida"
links_relatorios = []
for row in rows:
    a_tag = row.find('a', text='Relatório da Partida')
    if a_tag:
        href = a_tag.get('href')
        if href and href != "#":  # Ignora links vazios ou "#" como marcador
            links_relatorios.append(href)
        else:
            links_relatorios.append(None)
    else:
        links_relatorios.append(None)

# Remove linhas onde a coluna "Relatório da Partida" é "Confronto"
tabela = tabela[~tabela['Relatório da Partida'].str.contains("Confronto", na=False)]

# Adiciona a coluna de links na tabela
tabela['Link Relatório'] = links_relatorios[:len(tabela)]  # Ajusta o tamanho da lista para o tamanho da tabela

C:\Users\eduar\AppData\Local\Temp\ipykernel_27540\192175195.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = row.find('a', text='Relatório da Partida')


In [6]:
# Desloca a coluna "Link Relatório" uma linha para cima
tabela['Link Relatório'] = tabela['Link Relatório'].shift(-1)

# Preenche a última linha da coluna "Link Relatório" com NaN (caso não esteja automaticamente)
tabela.loc[tabela.index[-1], 'Link Relatório'] = None


In [7]:
# Remove linhas onde "Link Relatório" é None
tabela = tabela[tabela['Link Relatório'].notna()]

In [8]:
tabela

,Sem,Dia,Data,Horário,Em casa,xG,Resultado,xG,Visitante,Público,Local,Árbitro,Relatório da Partida,Notas,Link Relatório
0,1,sáb,2024-08-17,18:30,Genoa,2.1,2–2,1.9,Inter,,Stadio Comunale Luigi Ferraris,,Relatório da Partida,,/pt/partidas/5dd9a8ae/Genoa-Internazionale-202...
1,1,sáb,2024-08-17,18:30,Parma,1.7,1–1,0.7,Fiorentina,,Stadio Ennio Tardini,,Relatório da Partida,,/pt/partidas/99186416/Parma-Fiorentina-2024Ago...
2,1,sáb,2024-08-17,20:45,Milan,2.1,2–2,1.0,Torino,,Stadio Giuseppe Meazza,,Relatório da Partida,,/pt/partidas/6321e075/Milan-Torino-2024Agosto1...
3,1,sáb,2024-08-17,20:45,Empoli,0.5,0–0,0.3,Monza,,Stadio Carlo Castellani - Computer Gross...,,Relatório da Partida,,/pt/partidas/e1942888/Empoli-Monza-2024Agosto1...
4,1,dom,2024-08-18,18:30,Hellas Verona,1.7,3–0,1.0,Napoli,,Stadio Marc'Antonio Bentegodi,,Relatório da Partida,,/pt/partidas/6a1d7fbe/Hellas-Verona-Napoli-202...
5,1,dom,2024-08-18,18:30,Bologna,3.2,1–1,0.9,Udinese,,Stadio Renato Dall'Ara,,Relatório da Partida,,/pt/partidas/a538d1ae/Bologna-Udinese-2024Agos...
6,1,dom,2024-08-18,20:45,Cagliari,1.0,0–0,0.9,Roma,,Unipol Domus,,Relatório da Partida,,/pt/partidas/8314db9d/Cagliari-Roma-2024Agosto...
7,1,dom,2024-08-18,20:45,Lazio,2.4,3–1,0.6,Venezia,,Stadio Olimpico,,Relatório da Partida,,/pt/partidas/8c25dfec/Lazio-Venezia-2024Agosto...
8,1,seg,2024-08-19,18:30,Lecce,1.5,0–4,1.7,Atalanta,,Stadio Comunale Ettore Giardiniero,,Relatório da Partida,,/pt/partidas/5b248995/Lecce-Atalanta-2024Agost...
9,1,seg,2024-08-19,20:45,Juventus,1.2,3–0,0.2,Como,,Allianz Stadium,,Relatório da Partida,,/pt/partidas/888690a6/Juventus-Como-2024Agosto...


In [9]:
tabela.to_csv('C://Users//eduar//Desktop//DEV//Football//SerieA//links.csv', index= False)